In [2]:
import numpy as np
import pandas as pd
import pyper

In [3]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = pd.DataFrame(np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]]))
X.columns = ["feat1","feat2","feat3","feat4","feat5","feat6","feat7","feat8","feat9","feat10","feat11","feat12","feat13"]
y = raw_df.values[1::2, 2]

In [4]:
X

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,feat11,feat12,feat13
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [5]:
r = pyper.R(use_pandas='True')

In [6]:
X_TRAIN = X.iloc[:350]
Y_TRAIN = y[:350]
X_VAL = X.iloc[350:450]
Y_VAL = y[350:450]
X_TEST = X.iloc[450:]
Y_TEST = y[450:]

In [7]:
r.assign("X_TRAIN", X_TRAIN)
r.assign("Y_TRAIN", Y_TRAIN)
r.assign("X_VAL", X_VAL)
r.assign("Y_VAL", Y_VAL)
r.assign("X_TEST", X_TEST)

r.assign("NTREE", 500)
r.assign("NODESIZE", 7)

In [8]:
a = r("source(file='rrf.R')")

In [9]:
a

"try({source(file='rrf.R')})\nRRF 1.9.1\nType rrfNews() to see new features/changes/bug fixes.\n     |      Out-of-bag   |       Test set    |\nTree |      MSE  %Var(y) |      MSE  %Var(y) |\n   1 |    35.92    50.18 |    115.2   126.41 |\n   2 |    22.69    31.69 |    77.51    85.03 |\n   3 |    17.19    24.01 |    69.22    75.94 |\n   4 |    15.17    21.19 |    64.04    70.25 |\n   5 |     13.6    18.99 |    58.04    63.67 |\n   6 |    13.35    18.65 |    56.59    62.08 |\n   7 |    12.03    16.81 |    57.15    62.70 |\n   8 |    12.11    16.91 |    66.25    72.68 |\n   9 |    11.64    16.26 |     68.9    75.58 |\n  10 |     10.3    14.39 |    70.17    76.98 |\n  11 |    10.09    14.09 |    68.42    75.06 |\n  12 |    9.818    13.71 |    68.29    74.91 |\n  13 |    9.742    13.61 |    69.43    76.17 |\n  14 |    9.449    13.20 |    68.12    74.73 |\n  15 |    9.022    12.60 |    66.74    73.21 |\n  16 |    8.658    12.09 |    67.65    74.22 |\n  17 |    8.496    11.87 |    67.98    7